## Análisis Comercial de clientes

In [ ]:
# 1. Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px

# Configuración visual
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="muted", font_scale=1.1)


## Carga de Datos

In [ ]:
# Cargar desde CSV (puedes adaptarlo para leer desde BD o Excel)
# El CSV debería tener columnas como: Cliente, Licencias, Productos, Tickets, Llamadas, Antiguedad_Anios
df = pd.read_csv("clientes.csv")

print("Vista previa de datos:")
display(df.head())
print("\nInformación general:")
print(df.info())



In [ ]:
# =====================================================
# 3. LIMPIEZA Y PREPARACIÓN
# =====================================================
# Eliminar duplicados
df = df.drop_duplicates()

# Manejo de valores nulos
df = df.fillna(0)

# Crear métricas derivadas
df["Solicitudes_Total"] = df["Tickets"] + df["Llamadas"]
df["Solicitudes_por_Licencia"] = df["Solicitudes_Total"] / df["Licencias"].replace(0, np.nan)
df["Productos_por_Año"] = df["Productos"] / df["Antiguedad_Anios"].replace(0, np.nan)



In [ ]:
# =====================================================
# 4. ANÁLISIS EXPLORATORIO
# =====================================================

# Estadísticas descriptivas
print("\nEstadísticas generales:")
display(df.describe())

# Distribución de clientes por licencias
plt.figure(figsize=(8,5))
sns.histplot(df["Licencias"], bins=20, kde=True)
plt.title("Distribución de Licencias por Cliente")
plt.show()

# Relación Licencias vs. Tickets
plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x="Licencias", y="Tickets")
plt.title("Licencias vs Tickets")
plt.show()

# Pareto: clientes que más solicitudes generan
df_pareto = df.sort_values("Solicitudes_Total", ascending=False)
df_pareto["% Acumulado"] = df_pareto["Solicitudes_Total"].cumsum() / df_pareto["Solicitudes_Total"].sum() * 100

fig = px.bar(df_pareto, x="Cliente", y="Solicitudes_Total", title="Pareto de Solicitudes por Cliente")
fig.add_scatter(x=df_pareto["Cliente"], y=df_pareto["% Acumulado"], mode="lines+markers", name="% Acumulado")
fig.show()



In [ ]:
# =====================================================
# 5. SEGMENTACIÓN DE CLIENTES (CLUSTERING)
# =====================================================

# Seleccionar variables para el modelo
vars_modelo = ["Licencias", "Productos", "Solicitudes_Total", "Antiguedad_Anios"]
X = df[vars_modelo]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Visualización de clusters
fig = px.scatter_3d(df, x="Licencias", y="Productos", z="Solicitudes_Total",
                    color="Cluster", hover_data=["Cliente"])
fig.show()



In [ ]:
# =====================================================
# 6. INSIGHTS INICIALES
# =====================================================
# Agrupar por cluster para ver promedios
insights = df.groupby("Cluster")[vars_modelo].mean()
print("\nPromedios por cluster:")
display(insights)

# Guardar resultados
df.to_csv("clientes_segmentados.csv", index=False)
print("\nArchivo 'clientes_segmentados.csv' generado.")